# __Predicting Outcomes of Call Option Contracts__

# SECTION 1: Preparation

##  1.1.) Loading the python packages

In [1]:

from sklearn import datasets

# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
import random

# Load libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV
from evolutionary_search import EvolutionaryAlgorithmSearchCV

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier

#importing classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

#from sklearn.metrics import balanced_accuracy_score
#from sklearn.metrics import confusion_matrix
#from imblearn.metrics import classification_report_imbalanced

#Libraries for Saving the Model
from pickle import dump
from pickle import load

import warnings
warnings.filterwarnings('ignore')



## 1.2.) Loading the Data

In [2]:
#NOTE: This iris dataset is for debugging purposes only

#iris = datasets.load_iris()

#X = iris.data

#y = iris.target

#validation_size = 0.2

#seed = 7

In [3]:
#Import call options info

df = pd.read_csv(
    Path("../Resources/full_df_filt.csv")
)


In [4]:
df.head()

,CONTRACT,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE DISTANCE,INFLATION%,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,474.0 2022-01-07,476.43,1/7/2022,4.27,0.62598,0.05055,0.21136,-0.28773,0.03788,0.13515,...,-2.43,0.8,0.55,-87.089202,0.0,17.68,0.196649,0.0258,0,0
1,491.0 2022-01-07,476.43,1/7/2022,4.27,0.01171,0.00474,0.01717,-0.01484,0.00056,0.11400,...,14.57,0.8,0.01,0.000000,2.0,17.68,0.196649,0.0258,0,0
2,490.0 2022-01-07,476.43,1/7/2022,4.27,0.01569,0.00635,0.02257,-0.01973,0.00132,0.11196,...,13.57,0.8,0.02,0.000000,2.0,17.68,0.196649,0.0258,0,0
3,489.0 2022-01-07,476.43,1/7/2022,4.27,0.02331,0.00888,0.03151,-0.02878,0.00105,0.11290,...,12.57,0.8,0.01,-66.666667,0.0,17.68,0.196649,0.0258,0,0
4,488.0 2022-01-07,476.43,1/7/2022,4.27,0.02852,0.01072,0.03713,-0.03303,0.00202,0.10768,...,11.57,0.8,0.01,-75.000000,0.0,17.68,0.196649,0.0258,0,0


# SECTION 2: Exploratory Data Analysis

## 2.1.) Descriptive Statistics

In [5]:
df.shape

(9276, 24)

In [6]:
#Check for any null values and remove the null values
print('Null Values =', df.isnull().values.any())

#Drop NaNs
df.dropna()

Null Values = True


,CONTRACT,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE DISTANCE,INFLATION%,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,474.0 2022-01-07,476.43,1/7/2022,4.27,0.62598,0.05055,0.21136,-0.28773,0.03788,0.13515,...,-2.43,0.8,0.55,-87.089202,0.0,17.68,0.196649,0.0258,0,0
1,491.0 2022-01-07,476.43,1/7/2022,4.27,0.01171,0.00474,0.01717,-0.01484,0.00056,0.11400,...,14.57,0.8,0.01,0.000000,2.0,17.68,0.196649,0.0258,0,0
2,490.0 2022-01-07,476.43,1/7/2022,4.27,0.01569,0.00635,0.02257,-0.01973,0.00132,0.11196,...,13.57,0.8,0.02,0.000000,2.0,17.68,0.196649,0.0258,0,0
3,489.0 2022-01-07,476.43,1/7/2022,4.27,0.02331,0.00888,0.03151,-0.02878,0.00105,0.11290,...,12.57,0.8,0.01,-66.666667,0.0,17.68,0.196649,0.0258,0,0
4,488.0 2022-01-07,476.43,1/7/2022,4.27,0.02852,0.01072,0.03713,-0.03303,0.00202,0.10768,...,11.57,0.8,0.01,-75.000000,0.0,17.68,0.196649,0.0258,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9271,450.0 2022-03-25,443.84,3/25/2022,2.00,0.17749,0.03974,0.09998,-0.31313,0.00589,0.17096,...,6.16,0.8,0.63,0.000000,2.0,23.47,0.143046,0.0000,0,0
9272,446.0 2022-03-25,443.84,3/25/2022,2.00,0.38057,0.05533,0.14627,-0.52994,0.01244,0.17937,...,2.16,0.8,1.82,0.000000,2.0,23.47,0.143046,0.0000,0,0
9273,447.0 2022-03-25,443.84,3/25/2022,2.00,0.32357,0.05317,0.13772,-0.48289,0.01081,0.17641,...,3.16,0.8,1.43,0.000000,2.0,23.47,0.143046,0.0000,0,0
9274,448.0 2022-03-25,443.84,3/25/2022,2.00,0.27055,0.04956,0.12697,-0.42935,0.00908,0.17361,...,4.16,0.8,1.10,0.000000,2.0,23.47,0.143046,0.0000,0,0


In [7]:
display(df.head())
df.tail()

,CONTRACT,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE DISTANCE,INFLATION%,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,474.0 2022-01-07,476.43,1/7/2022,4.27,0.62598,0.05055,0.21136,-0.28773,0.03788,0.13515,...,-2.43,0.8,0.55,-87.089202,0.0,17.68,0.196649,0.0258,0,0
1,491.0 2022-01-07,476.43,1/7/2022,4.27,0.01171,0.00474,0.01717,-0.01484,0.00056,0.11400,...,14.57,0.8,0.01,0.000000,2.0,17.68,0.196649,0.0258,0,0
2,490.0 2022-01-07,476.43,1/7/2022,4.27,0.01569,0.00635,0.02257,-0.01973,0.00132,0.11196,...,13.57,0.8,0.02,0.000000,2.0,17.68,0.196649,0.0258,0,0
3,489.0 2022-01-07,476.43,1/7/2022,4.27,0.02331,0.00888,0.03151,-0.02878,0.00105,0.11290,...,12.57,0.8,0.01,-66.666667,0.0,17.68,0.196649,0.0258,0,0
4,488.0 2022-01-07,476.43,1/7/2022,4.27,0.02852,0.01072,0.03713,-0.03303,0.00202,0.10768,...,11.57,0.8,0.01,-75.000000,0.0,17.68,0.196649,0.0258,0,0


,CONTRACT,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE DISTANCE,INFLATION%,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
9271,450.0 2022-03-25,443.84,3/25/2022,2.0,0.17749,0.03974,0.09998,-0.31313,0.00589,0.17096,...,6.16,0.8,0.63,0.0,2.0,23.47,0.143046,0.0,0,0
9272,446.0 2022-03-25,443.84,3/25/2022,2.0,0.38057,0.05533,0.14627,-0.52994,0.01244,0.17937,...,2.16,0.8,1.82,0.0,2.0,23.47,0.143046,0.0,0,0
9273,447.0 2022-03-25,443.84,3/25/2022,2.0,0.32357,0.05317,0.13772,-0.48289,0.01081,0.17641,...,3.16,0.8,1.43,0.0,2.0,23.47,0.143046,0.0,0,0
9274,448.0 2022-03-25,443.84,3/25/2022,2.0,0.27055,0.04956,0.12697,-0.42935,0.00908,0.17361,...,4.16,0.8,1.10,0.0,2.0,23.47,0.143046,0.0,0,0
9275,445.0 2022-03-25,443.84,3/25/2022,2.0,0.43827,0.05632,0.15118,-0.56141,0.01461,0.18219,...,1.16,0.8,2.30,0.0,2.0,23.47,0.143046,0.0,0,0


In [8]:
df.describe()

,SPY PRICE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,...,STRIKE DISTANCE,INFLATION%,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
count,9276.000000,9276.000000,9276.000000,9276.000000,9276.000000,9276.000000,9276.000000,9275.000000,9276.000000,9276.000000,...,9276.000000,9276.000000,9276.000000,9276.000000,9276.000000,9276.000000,9276.000000,9276.000000,9276.000000,9276.000000
mean,445.466729,3.135291,0.336379,0.028479,0.130157,-0.406592,0.013854,0.245399,4595.604894,3.138607,...,5.064319,0.625873,3.635285,18.637964,2.039996,25.764125,0.029203,-0.002008,-0.027167,-0.027167
std,16.464778,0.819373,0.230123,0.013810,0.053677,0.226454,0.009278,0.084728,6716.052305,3.151040,...,7.820791,0.124095,4.320817,117.170459,1.807268,5.446261,0.115361,0.054477,0.162578,0.162578
min,415.920000,2.000000,0.001990,0.000000,0.000000,-1.063160,-0.000350,0.089130,0.000000,0.010000,...,-18.400000,0.500000,0.010000,-97.701149,0.000000,16.520000,-0.206529,-0.277600,-1.000000,-1.000000
25%,432.080000,2.210000,0.139185,0.019860,0.093575,-0.552365,0.006320,0.180655,739.000000,0.650000,...,-0.422500,0.500000,0.220000,-65.204921,0.000000,21.860000,-0.072222,0.000000,0.000000,0.000000
50%,445.190000,3.150000,0.309215,0.026970,0.142825,-0.415375,0.012690,0.240220,2255.500000,2.120000,...,5.060000,0.600000,1.760000,-6.914361,2.000000,25.260000,0.072969,0.000000,0.000000,0.000000
75%,456.820000,4.060000,0.505085,0.036373,0.172192,-0.239032,0.020222,0.301420,5625.000000,4.700000,...,10.562500,0.800000,5.900000,54.619012,3.000000,30.560000,0.120354,0.000000,0.000000,0.000000
max,479.650000,4.270000,1.000000,0.088050,0.221910,0.000000,0.040320,0.495160,63071.000000,19.370000,...,29.620000,0.800000,18.470000,912.500000,5.000000,37.980000,0.196649,0.177900,0.000000,0.000000


In [9]:
df.dtypes

CONTRACT                object
SPY PRICE              float64
EXPIRE_DATE             object
DTE                    float64
C_DELTA                float64
C_GAMMA                float64
C_VEGA                 float64
C_THETA                float64
C_RHO                  float64
C_IV                   float64
C_VOLUME                 int64
C_LAST                 float64
STRIKE                   int64
STRIKE_DISTANCE_PCT    float64
STRIKE DISTANCE        float64
INFLATION%             float64
PRICECLOSE             float64
ROI %                  float64
y                      float64
VIX PRICE              float64
AVG SCORE              float64
MED SCORE              float64
SEN IND AVG NUM          int64
SEN IND MED NUM          int64
dtype: object

## 2.2.) Feature Analysis and Exploration

### Plotting features according to contract length

In [10]:
#
#contract_outcome = df.groupby('STRIKE_DISTANCE_PCT')['y'].value_counts(normalize=True).loc[:,1]
#sns.set(rc={'figure.figsize':(12,5)})
#sns.barplot(x=contract_outcome.index, y=contract_outcome.values, color='#5975A4', saturation=1)

### Eliminate Uncorrelated Features

In [11]:
#Calculate correlation of each feature with 'y'

correlation = df.corr()
correlation_df = abs(correlation['y'])

In [12]:
correlation_df.sort_values(ascending=False)

y                      1.000000
ROI %                  0.868386
PRICECLOSE             0.569380
SEN IND AVG NUM        0.253530
STRIKE                 0.220134
SPY PRICE              0.166758
C_VEGA                 0.160804
C_RHO                  0.133912
DTE                    0.133716
C_DELTA                0.133436
C_LAST                 0.127132
C_IV                   0.118368
STRIKE DISTANCE        0.115619
C_VOLUME               0.106989
VIX PRICE              0.101851
STRIKE_DISTANCE_PCT    0.081797
C_THETA                0.077824
SEN IND MED NUM        0.050243
MED SCORE              0.024780
INFLATION%             0.018557
C_GAMMA                0.014040
AVG SCORE              0.012744
Name: y, dtype: float64

In [13]:
#Drop variables with less than 3% correlation with contract outcome ('y')

#drop_list_corr = sorted(list(correlation_df[correlation_df < 0.015].index))
#print(drop_list_corr)

In [14]:
#Dropping because C_IV convolutes outcome for some buggish reason

df.drop(labels=["C_IV"], axis=1, inplace=True)

### Drop Columns Not Needed for Machine Learning

In [15]:
df.drop(columns=["CONTRACT"], inplace=True)

In [16]:
df.drop(columns=["ROI %"], inplace=True)

In [17]:
df.drop(columns=["PRICECLOSE"], inplace=True)

In [23]:
df.drop(columns=["EXPIRE_DATE"], inplace=True)

In [ ]:
df.drop(columns=["C_VOLUME"], inplace=True)

# SECTION 3: Evaluate Algorithms and Models

## 3.1.) Train Test Split and Evaluation Metrics

In [24]:
#Split into training and testing groups

#Split out validation dataset for the end
y = df["y"]

X = df.drop(columns='y')


In [25]:
y.value_counts()

0.0    2493
1.0    2008
5.0    1640
3.0    1321
2.0    1252
4.0     562
Name: y, dtype: int64

In [26]:

validation_size = 0.2

seed = 7

X_train, X_validation, y_train, y_validation = train_test_split(X, y, stratify=y, test_size=validation_size, random_state=seed)


In [27]:
X_validation

,SPY PRICE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_VOLUME,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,INFLATION%,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
6376,434.68,4.15,0.46640,0.02684,0.19961,-0.62310,0.02562,2471,5.09,436,0.003,1.32,0.6,30.64,-0.042974,0.0000,0,0
6973,438.71,2.02,0.19950,0.03049,0.10608,-0.44111,0.00640,6025,1.02,446,0.017,7.29,0.6,30.32,-0.128163,0.0000,0,0
8631,445.53,4.15,0.10361,0.02027,0.09285,-0.15747,0.00633,691,0.44,457,0.026,11.47,0.8,23.37,0.118401,0.0000,0,0
9174,445.07,2.10,0.02164,0.00772,0.02090,-0.04467,0.00059,1243,0.05,459,0.031,13.93,0.8,23.67,0.143046,0.0000,0,0
7498,416.38,3.00,0.02227,0.00413,0.02269,-0.06798,0.00077,818,0.11,442,0.062,25.62,0.6,35.33,-0.167307,0.0000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3932,457.40,2.02,0.66551,0.04587,0.12464,-0.40401,0.00930,1960,6.32,452,0.012,-5.40,0.5,20.61,0.165217,0.1779,0,0
8245,425.77,3.02,0.45003,0.02882,0.16571,-0.59044,0.01495,5497,4.80,426,0.001,0.23,0.8,29.92,-0.011496,0.0000,0,0
275,479.65,3.25,0.02342,0.01047,0.02781,-0.02828,0.00113,283,0.04,490,0.022,10.35,0.8,16.85,0.120354,0.0000,0,0
5244,444.19,2.27,0.36255,0.03920,0.15086,-0.57586,0.01273,1,2.34,447,0.006,2.81,0.6,26.58,0.025877,0.0000,0,0


In [28]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_validation_scaled = X_scaler.transform(X_validation)

## 3.2.) Quick Check of Models and Algorithms

In [29]:
#Spot check Classification algorithms

models = []

#Boosting methods

models.append(('XGB', XGBClassifier()))
models.append(('AB', AdaBoostClassifier()))
#models.append(('GBM', GradientBoostingClassifier()))

#Bagging methods

models.append(('RF', RandomForestClassifier()))
models.append(('ET', ExtraTreesClassifier()))

#SVC 
#models.append(('SVC', SVC()))

#K Nearest Neighbors
models.append(('KNN', KNeighborsClassifier()))


In [30]:
#Test options for classification

num_folds = 10
seed = 7

In [31]:
results = []

names = []

scoring = 'accuracy'

for name, model in models:
    kfold = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    findings_summary = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(findings_summary)

XGB: 0.920350 (0.011267)
AB: 0.604582 (0.020600)
RF: 0.914286 (0.013262)
ET: 0.917655 (0.011710)
KNN: 0.820620 (0.009894)


In [ ]:
#Compare the model outcomes

fig = pyplot.figure()
fig.suptitle('Model Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
fig.set_size_inches(8,4)
pyplot.show()

## 3.3.) Selecting Random Forest (or whichver tree method best), Calculating its Baseline 

In [ ]:
#Estimate accuracy on training set

rf_model = RandomForestClassifier()

cv = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)
scores = cross_val_score(rf_model, X_train_scaled, y_train, scoring='accuracy', cv=cv)
avg_score = np.mean(scores)
avg_score

In [ ]:
#Estimate accuracy on validation set

rf_model = RandomForestClassifier()

rf_model.fit(X_train_scaled, y_train)

predictions = rf_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Visualization of the Confusion Matrix

df_matrix = pd.DataFrame(confusion_matrix(y_validation, predictions), columns=np.unique(y_validation), index = np.unique(y_validation))
df_matrix.index.name = 'Actual'
df_matrix.columns.name = 'Predicted'
sns.heatmap(df_matrix, cmap="Blues", annot=True, annot_kws={"size": 16}) 

# SECTION 4: Model Tuning 

## 4.1.) Hyperparameter Tuning for Random Forest

### APPROACH (A) 
## __*(CAUTION TAKES A LONG TIME!)*__

In [ ]:


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {}

paramgrid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,}
              # 'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
              # 'bootstrap': bootstrap}

random.seed(1)

cv = EvolutionaryAlgorithmSearchCV(estimator=RandomForestClassifier(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=4),
                                   verbose=1,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=4)
cv.fit(X_train_scaled, y_train)

### APPROACH (B)
## __*(CAUTION TAKES A LONG TIME!)*__

In [ ]:
# Grid Search: (select model) Tuning

n_estimators = [20, 100, 180, 1000]
max_features = [1, 2, 3, 4]

param_grid = dict(n_estimators=n_estimators, max_features=max_features)

rf_model = RandomForestClassifier()

kfold = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)

grid = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X_train_scaled, y_train)

#Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
ranks = grid_result.cv_results_['rank_test_score']
for mean, stdev, param, rank in zip(means, stds, params, ranks):
    print("#%d %f (%f) with: %r" % (rank, mean, stdev, param))

### PROCEEDING WITH BEST APPROACH

In [ ]:
#Prepare the model with parameters decided in previous cell

rf_model = RandomForestClassifier() #(n_estimators=200, max_features="sqrt", max_depth=100, min_samples_split=5, min_samples_leaf=1, bootstrap=False)
rf_model.fit(X_train_scaled, y_train)

In [ ]:
#Score predictions of training set

training_predictions = rf_model.predict(X_train_scaled)

print(accuracy_score(y_train, training_predictions))

print('Micro Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_train, training_predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_train, training_predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_train, training_predictions, average='weighted')))


print(classification_report(y_train, training_predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

In [ ]:
#Score predictions of validation set

predictions = rf_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Micro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))


print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Visualization of the Confusion Matrix

df_matrix = pd.DataFrame(confusion_matrix(y_validation, predictions), columns=np.unique(y_validation), index = np.unique(y_validation))
df_matrix.index.name = 'Actual'
df_matrix.columns.name = 'Predicted'
sns.heatmap(df_matrix, cmap="Blues", annot=True, annot_kws={"size": 16}) 

## 4.2.) Determining Feature Importance

In [ ]:
#Feature importances

importances = rf_model.feature_importances_

In [ ]:
important_features = zip(X.columns, rf_model.feature_importances_)

In [ ]:
importances_df = pd.DataFrame(important_features)

In [ ]:
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

In [ ]:
importances_df = importances_df.set_index('Feature')

In [ ]:
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
importances_df

In [ ]:
#Plot the top 10 most important features

importances_df[0:10].plot(
    kind='barh',
    color='green',
    title='Feature Importance',
    legend=True)

## 4.2. Bayesian Optimization with HYPEROPT for XGB

## __*(CAUTION TAKES A COUPLE OF MIN!)*__

In [ ]:
#Initialize domain space for range of values 
 
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1, 9),
        'reg_alpha' : hp.quniform('reg_alpha', 40, 180, 1),
        'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }


The available hyperopt optimization algorithms are -

hp.choice(label, options) — Returns one of the options, which should be a list or tuple.

hp.randint(label, upper) — Returns a random integer between the range [0, upper).

hp.uniform(label, low, high) — Returns a value uniformly between low and high.

hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer.

hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.

In [ ]:
def objective(space):
    xgb_model = xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [(X_train_scaled, y_train), (X_validation_scaled, y_validation)]
    
    xgb_model.fit(X_train_scaled, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = xgb_model.predict(X_validation_scaled)
    accuracy = accuracy_score(y_validation, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [ ]:
#Establish XGB model instance
#Fitting training dataset to model

xgb_model = xgb.XGBClassifier() #colsample_bytree=0.8056144829851541, max_depth=8, gamma=6.979438878191186, min_child_weight=10.0, reg_lambda=0.00013734197899190276, reg_alpha=0)
xgb_model.fit(X_train_scaled, y_train)

In [ ]:
#Predicting on training set

training_predictions = xgb_model.predict(X_train_scaled)

print(accuracy_score(y_train, training_predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_train, training_predictions, average='weighted')))

print(classification_report(y_train, training_predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

In [ ]:
#Predicting on validation set

predictions = xgb_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Get probabilities of predictions

pred_proba = xgb_model.predict_proba(X_validation_scaled)
pred_proba

In [ ]:
proba_df = pd.DataFrame(pred_proba.round(2))

In [ ]:
#Formatting probabilities DataFrame

proba_df["Actual"] = y_validation.reset_index(drop=True)
proba_df.columns = ["'Strong Sell' Probability", "'Sell' Probability", "'Pass' Probability", "'Buy' Probability", "'Strong Buy' Probability", "'Very High Return' Prob.", "Actual"]
proba_df

In [ ]:
#Labeling caterogires

proba_df.loc[proba_df["Actual"] == 0, "Actual"] = "Strong Sell"
proba_df.loc[proba_df["Actual"] == 1, "Actual"] = "Sell"
proba_df.loc[proba_df["Actual"] == 2, "Actual"] = "Pass"
proba_df.loc[proba_df["Actual"] == 3, "Actual"] = "Buy"
proba_df.loc[proba_df["Actual"] == 4, "Actual"] = "Strong Buy"
proba_df.loc[proba_df["Actual"] == 5, "Actual"] = "Very High Return"

proba_df.head(50)

In [ ]:
#Get ROC AUC score

roc_auc_score(y_validation, pred_proba, multi_class="ovr")

## 4.2.) Determining Feature Importance for XGBoost Model

In [ ]:
#Feature importances

importances = xgb_model.feature_importances_

In [ ]:
important_features = zip(X.columns, xgb_model.feature_importances_)

In [ ]:
importances_df = pd.DataFrame(important_features)

In [ ]:
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

In [ ]:
importances_df = importances_df.set_index('Feature')

In [ ]:
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
importances_df

In [ ]:
#Plot the top 10 most important features

importances_df[0:10].plot(
    kind='barh',
    color='green',
    title='Feature Importance',
    legend=True)

In [ ]:
#et_model = ExtraTreesClassifier()
#et_model.fit(X_train_scaled, y_train)

In [ ]:
#ab_model = AdaBoostClassifier()
#ab_model.fit(X_train_scaled, y_train)

In [ ]:
#gb_model = GradientBoostingClassifier()
#ab_model.fit(X_train_scaled, y_train)